# Load libraries

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn import metrics
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool
from src.sampling_methods.sampler import DataSampler
from typing import Optional, Literal

import warnings
warnings.filterwarnings("ignore")

In [2]:
def check_result(model,X_test,y_test):
  metrics_dict = {}
  fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict(X_test))
  auc_score = metrics.auc(fpr, tpr)
  f1_sc = metrics.f1_score(y_test, model.predict(X_test), average='macro')
  metrics_dict['auc_score'] = auc_score
  metrics_dict['f1_score'] = f1_sc
  return metrics_dict

In [3]:
def search_num_features(df, feature_importance, upsamp_func: Optional[Literal['ROS', 'RUS', 'SMOTE', 'OSS']] = None, p = 0.3, step = 5):
  best_score = 0
  best_num_features = 0

  for num_col in tqdm(range(1, len(feature_importance), step)):
    features = list(feature_importance.iloc[:num_col,:]['feature_names'])
    data = df[features + ['Machine failure']]
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = ['Machine failure']),
                                                        data['Machine failure'],
                                                        test_size=0.33,
                                                        random_state=42,
                                                        stratify = df['Machine failure'])
    
    if upsamp_func != 'ROS':
      X_train, y_train = DataSampler.ROS(X_train, y_train, p, separation = True)
    elif upsamp_func != 'RUS':
      X_train, y_train = DataSampler.RUS(X_train, y_train, p, separation = True)
    elif upsamp_func != 'SMOTE':
      X_train, y_train = DataSampler.SMOTE(X_train, y_train, p, separation = True)
    elif upsamp_func != 'OSS':
      X_train, y_train = DataSampler.OSS(X_train, y_train, separation = True)
    
    train_pool = Pool(data=X_train, label=y_train)
    CatBoost = CatBoostClassifier(verbose=False,random_seed=42)
    CatBoost.fit(train_pool)
    metrics_dict = check_result(CatBoost, X_test, y_test)
    print(f'F1_score - {metrics_dict["f1_score"]}, num_features - {best_num_features}, AUC_score = {metrics_dict["auc_score"]}')
    if metrics_dict['f1_score'] > best_score:
      best_score = metrics_dict['f1_score']
      best_num_features = num_col
  print(f'Best F1_score - {best_score}, num_features - {best_num_features}')


# Read data

In [4]:
df = pd.read_csv('../data/prepared_data.csv')
df_fe = pd.read_csv('../data/data_fe.csv')

# Train Catboost

## Test data without upsampling

### Find feature importance

In [5]:
CatBoost = CatBoostClassifier(verbose=False,random_seed=42)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df_fe.drop(columns = ['Machine failure']),
                                                    df_fe['Machine failure'],
                                                    test_size=0.33,
                                                    random_state=42,
                                                    stratify = df['Machine failure'])

train_pool = Pool(data=X_train, label=y_train)

In [7]:
CatBoost.fit(train_pool)

feature_importance = pd.DataFrame({'feature_importance': CatBoost.get_feature_importance(train_pool), 
              'feature_names': X_test.columns}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
              
check_result(CatBoost, X_test, y_test )

{'auc_score': 0.9003741709983868, 'f1_score': 0.9241142395957508}

In [8]:
feature_importance.to_csv('../data/feature_importance/feature_importance.csv', index = False)

### Find best amount of features

In [9]:
search_num_features(df_fe,feature_importance, step = 5)

  4%|▍         | 1/25 [00:02<00:55,  2.33s/it]

F1_score - 0.6549366900395834, num_features - 0, AUC_score = 0.6799605664097508


  8%|▊         | 2/25 [00:04<00:55,  2.42s/it]

F1_score - 0.790241794004729, num_features - 1, AUC_score = 0.7877419788492562


 12%|█▏        | 3/25 [00:07<00:55,  2.52s/it]

...

F1_score - 0.9393574536305039, num_features - 66, AUC_score = 0.9185449901416025


 96%|█████████▌| 24/25 [02:10<00:07,  7.98s/it]

F1_score - 0.9399027517255195, num_features - 66, AUC_score = 0.9228524377128516


100%|██████████| 25/25 [02:19<00:00,  5.57s/it]

F1_score - 0.9442088573400635, num_features - 66, AUC_score = 0.9231661139989245
Best F1_score - 0.953911664759183, num_features - 66


## Test data with upsampling

### Find feature importance

In [10]:
CatBoost = CatBoostClassifier(verbose=False,random_seed=42)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df_fe.drop(columns = ['Machine failure']),
                                                    df_fe['Machine failure'],
                                                    test_size=0.33,
                                                    random_state=42,
                                                    stratify = df['Machine failure'])
X_train, y_train = DataSampler.ROS(X_train, y_train, 1, separation = True)

train_pool = Pool(data=X_train, label=y_train)

In [13]:
CatBoost.fit(train_pool)

feature_importance = pd.DataFrame({'feature_importance': CatBoost.get_feature_importance(train_pool), 
              'feature_names': X_test.columns}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
              
check_result(CatBoost, X_test, y_test)

{'auc_score': 0.9182313138555297, 'f1_score': 0.935094971863561}

In [14]:
feature_importance.to_csv('../data/feature_importance/feature_importance_ROS.csv', index = False)

### Find best amount of features

In [15]:
search_num_features(df_fe, feature_importance, upsamp_func = 'ROS', step = 5)

  4%|▍         | 1/25 [00:02<00:51,  2.14s/it]

F1_score - 0.6721066267795615, num_features - 0, AUC_score = 0.6149735615701739


  8%|▊         | 2/25 [00:04<00:54,  2.36s/it]

F1_score - 0.8975533368805821, num_features - 1, AUC_score = 0.8855697705682022


 12%|█▏        | 3/25 [00:07<00:58,  2.66s/it]

...

F1_score - 0.9296546462113844, num_features - 86, AUC_score = 0.9093027424269583


 96%|█████████▌| 24/25 [02:01<00:07,  7.74s/it]

F1_score - 0.926224010731053, num_features - 86, AUC_score = 0.9005310091414231


100%|██████████| 25/25 [02:09<00:00,  5.17s/it]

F1_score - 0.9372166333480656, num_features - 86, AUC_score = 0.9183881519985662
Best F1_score - 0.9409792085848425, num_features - 86
